# Example for grasping in Sim
* A grasp motion is designed using KOMO, where the 2-waypoint IK problem defines constraints over a sequence of two configurations, one for each waypoint.
* Each waypoint is here quite rigidly constrained by 6D endeff pose. For convenience, both endeff poses have been defined as 'marker' frames in the configuration. This is a common pattern: Define some convenient reference frames in the configuration, then define NLP constraints relative to these.
* The solver returns a sequence of two joint positions.
* Both are added to the BotOp spline pipeline (with hard timings 2 and 3 sec for them). They define a spline transitioning smoothly through both waypoints.
* The gripper is closed. (In simulation, a hard kinematic link is created!)
* After homing, the gripper is opened. (In simulation, the hard kinematic link is broken again; the object falls.)

In [1]:
from robotic import ry
import numpy as np
import time

In [2]:
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000., 'physx/angularDamping': 10., 'physx/defaultFriction': 100.})
ry.params_print()

-- ry.cpp:operator():101(0) python,
message: "Hello, this is been loaded",
LGP/cameraFocus: [1, 0.5, 1],
LGP/collisions: 1,
LGP/stopTime: 300,
LGP/stopSol: 6,
opt/maxStep: 0.1,
opt/verbose: 6,
opt/boundedNewton!,
botsim/engine: physx,
physx/multiBody!,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/angularDamping: 10,
physx/defaultFriction: 100


In [3]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False)

0

In [4]:
#   .setShape(ry.ST.sphere, size=[.025]) \
C.addFrame('box') \
    .setPosition([-.25,.1,.675]) \
    .setShape(ry.ST.ssBox, size=[.05,.05,.05,.005]) \
    .setColor([1,.5,0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

0

In [5]:
# WAYPOINT ENGINEERING:
# manually define frames as an endeff waypoints, relative to box:
way0 = C.addFrame('way0', 'box')
way1 = C.addFrame('way1', 'box')

In [6]:
way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(0 0 .1) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('d(90 0 0 1)')

C.view()

86

In [7]:
# define a 2 waypoint problem in KOMO
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'way0'], ry.OT.eq, [1e1]);
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1]);

In [8]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

====nlp==== method:AugmentedLagrangian bounded: no
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):93.1593 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    69.6169  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    47.7566  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    28.1096  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    11.4588  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    6.02281  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):     4.0943  ACCEPT
--newton-- it:   7  |Delta|:   0.170104  alpha:          1  evals:   8  f(y):    3.57494  ACCEPT
--newton-- it:   8  |Delta|:  0.0579509  alpha:          1  evals:   9  f(y):    3.52862  ACCEPT
--newton-- it:   9  |Delta|:  0.0238862  alpha:          1  evals:  10  f(y)

In [9]:
komo.view(False, "waypoints solution")

0

In [10]:
komo.view_close()
path = komo.getPath()

In [ ]:
bot = ry.BotOp(C, False)
bot.home(C)

In [ ]:
bot.home(C)

In [ ]:
bot.gripperOpen(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [ ]:
bot.move(path, [2., 3.])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [ ]:
bot.gripperCloseGrasp(ry._left, 'box')
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [ ]:
bot.home(C)

In [ ]:
bot.gripperOpen(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [ ]:
del bot

In [ ]:
del C

## Example for pushing in sim
* A push motion is designed using KOMO, there the 2-waypoint IK problem defines constraints over a sequence of two configurations, one for each waypoint.
* Each waypoint is here quite rigidly constrained by 6D endeff pose. For convenience, both endeff poses have been defined as 'marker' frames in the configuration. This is a common pattern: Define some convenient reference frames in the configuration, then define NLP constraints relative to these.
* The solver returns a sequence of two joint positions.
* Both are added to the BotOp spline pipeline. They define a linear spline interpolation with zero end velocity for both waypoints.

In [ ]:
from robotic import ry
import numpy as np
import time

In [ ]:
print('ry version', ry.__version__, ry.compiled())
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_print()

In [ ]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False)

In [ ]:
C.addFrame('box') \
    .setPosition([-.25,.1,.675]) \
    .setShape(ry.ST.ssBox, size=[.05,.05,.05,.005]) \
    .setColor([1,.5,0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

In [ ]:
# WAYPOINT ENGINEERING:
# manually define frames as an endeff waypoints, relative to box:
way0 = C.addFrame('way0', 'box')
way1 = C.addFrame('way1', 'box')

In [ ]:
way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(-.07 0 0) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('t(.2 0 0) d(90 0 0 1)')

C.view()

In [ ]:
# define a 2 waypoint problem in KOMO
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'way0'], ry.OT.eq, [1e1]);
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1]);

In [ ]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

In [ ]:
komo.view(False, "waypoints solution")

In [ ]:
komo.view_close()
path = komo.getPath()

In [ ]:
bot = ry.BotOp(C, False)
bot.home(C)

In [ ]:
bot.home(C)

In [ ]:
bot.gripperClose(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [ ]:
bot.moveTo(path[0])
bot.moveTo(path[1])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [ ]:
bot.home(C)

In [ ]:
bot.gripperOpen(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [ ]:
del bot

In [ ]:
del C